In [24]:
# imports
from csv import DictReader, DictWriter
from tqdm import tqdm
import pandas as pd
import numpy as np

In [25]:
original_column_names = ['station_id', 'longitude', 'latitude', 'time', 'AtmospherePressure', 'WindDirection', 'WindSpeed', 'Gust', 'WaveHeight', 'WavePeriod', 'MeanWaveDirection', 'Hmax', 'AirTemperature', 'DewPoint', 'SeaTemperature', 'RelativeHumidity']

new_column_names = ['AtmospherePressure', 'WindDirection', 'WindSpeed', 'Gust', 'AirTemperature', 'SeaTemperature', 'RelativeHumidity', 'WaveHeight', 'WavePeriod']

In [26]:
with open('../data/raw_data.csv', 'r') as f1, open('../data/interim_data.csv', 'w') as f2:
    reader = DictReader(f1, fieldnames=original_column_names)
    writer = DictWriter(f2, fieldnames=new_column_names, lineterminator='\n')

    next(reader)
    next(reader)

    writer.writeheader()

    for line in tqdm(reader):
        if any(line[column_name] == 'NaN' for column_name in new_column_names):
            continue
        writer.writerow({column_name: line[column_name] for column_name in new_column_names})

613392it [00:02, 270063.60it/s]


In [27]:
df = pd.read_csv('../data/interim_data.csv')
df.describe()

,AtmospherePressure,WindDirection,WindSpeed,Gust,AirTemperature,SeaTemperature,RelativeHumidity,WaveHeight,WavePeriod
count,385474.000000,385474.000000,385474.000000,385474.000000,385474.000000,385474.000000,385474.000000,385474.000000,385474.000000
mean,1013.791675,206.580169,15.119516,21.455321,11.606788,12.435129,82.288061,2.319344,6.144468
std,12.241422,89.416027,6.891260,9.537482,2.879286,2.299281,9.718917,1.517461,1.728072
min,910.300000,0.000000,0.000000,0.000000,0.264000,0.200000,0.391000,0.100000,2.000000
25%,1006.400000,150.000000,10.000000,14.800000,9.500000,10.700000,75.000000,1.200000,5.000000
50%,1015.039000,220.000000,15.000000,20.000000,11.600000,12.271000,83.000000,2.000000,6.000000
75%,1022.400000,270.000000,19.923000,27.000000,14.000000,14.300000,90.000000,3.000000,7.000000
max,1046.400000,360.000000,76.506000,200.000000,22.104000,24.200000,100.000000,14.100000,18.000000


In [28]:
# Use the interquartile range to remove the outliers in data.
for column_name in new_column_names:
    Q1 = df[column_name].quantile(0.25)
    Q3 = df[column_name].quantile(0.75)
    IQR = Q3 - Q1
    lower_whisker = Q1 - 1.5*IQR
    upper_whisker = Q3 + 1.5*IQR
    df = df[(df[column_name] >= lower_whisker) & (df[column_name] <= upper_whisker)]

df.describe()

,AtmospherePressure,WindDirection,WindSpeed,Gust,AirTemperature,SeaTemperature,RelativeHumidity,WaveHeight,WavePeriod
count,361891.000000,361891.000000,361891.000000,361891.000000,361891.000000,361891.000000,361891.000000,361891.000000,361891.000000
mean,1014.725728,204.739858,14.541314,20.516864,11.710947,12.498833,82.573291,2.102670,5.959147
std,11.323496,90.193495,6.320948,8.476560,2.878474,2.329885,9.596186,1.185541,1.548250
min,982.400000,0.000000,0.000000,0.000000,2.964000,5.900000,52.734000,0.100000,2.000000
25%,1007.400000,150.000000,10.000000,14.000000,9.600000,10.700000,76.000000,1.200000,5.000000
50%,1015.600000,217.000000,14.010000,20.000000,11.748000,12.400000,83.203000,1.900000,6.000000
75%,1022.656000,270.000000,19.000000,25.957000,14.100000,14.400000,90.000000,2.800000,7.000000
max,1046.400000,360.000000,33.300000,45.824000,20.600000,19.200000,100.000000,5.600000,10.000000


In [29]:
# Feature engineering
azimuth_rad = np.deg2rad(df['WindDirection'])
df['WindDirectionSin'] = np.sin(azimuth_rad)
df['WindDirectionCos'] = np.cos(azimuth_rad)
df = df.drop(columns=['WindDirection'])

df['TempDiff'] = df['AirTemperature'] - df['SeaTemperature']
df = df.drop(columns=['AirTemperature', 'SeaTemperature'])

df = df[['AtmospherePressure', 'WindSpeed', 'Gust', 'RelativeHumidity', 'WindDirectionSin', 'WindDirectionCos', 'TempDiff', 'WaveHeight', 'WavePeriod']]

In [30]:
df.to_csv('../data/processed_data.csv', index=False)